In [19]:
import yaml
import requests
from openai import OpenAI

In [24]:
INPUT_CREATOR_PROMPT = """
You are an agent made to extract locations, priorities, and area sizes from user requests. 
Priority is a number from 1 to 5, where 5 is the highest priority.
Area size is a number in square kilometers. This number depends of the size of the area requested. You are free to adapt this number to the area of the city, and adapt it with what the user asks for.
Please respect this output format:
requests:
  - location: "Tokyo"
    priority: 3
    area_size_km2: 100
  - location: "Montréal"
    priority: 2
    area_size_km2: 50
  - location: "Paris"
    priority: 1
"""

OUTPUT_CREATOR_PROMPT = """
Summarize satellite observation results. Describe orally the results of the observations. Mention your confidence in the results and excuses if the results is different from the user request.
This difference may be caused by a lot of factors, like the weather, the satellite position, or the area size.
"""

In [21]:
def get_gps_coordinates(location):
    url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json"
    response = requests.get(url, headers={"User-Agent": "satellite-scheduler"})
    data = response.json()
    if data:
        return float(data[0]["lat"]), float(data[0]["lon"])
    return None

def parse_user_request(llm, user_text):
    response = llm.chat.completions.create(
        messages=[{"role": "system", "content": INPUT_CREATOR_PROMPT},
                  {"role": "user", "content": user_text}],
        model="OpenAI.chatgpt-4o-latest",
        temperature=0.5
    )
    return yaml.safe_load(response.choices[0].message.content)

In [60]:

def generate_solver_input(locations):
    for loc in locations:
        gps_coordinates = get_gps_coordinates(loc["location"])
        loc["gps_coordinates"] = {"latitude": gps_coordinates[0], "longitude": gps_coordinates[1]} if gps_coordinates else None
        print(f"Location: {loc['location']}, GPS Coordinates: {loc['gps_coordinates']}")
    with open("solver_input.yaml", "w") as f:
        yaml.dump({"locations": locations}, f, encoding="utf-8", allow_unicode=True)

def simulate_solver():
    with open("solver_input.yaml", "r") as f:
        data = yaml.safe_load(f)
    output = {"observations": []}
    for req in data["locations"]:
        output["observations"].append({
            "location": req["location"],
            "success": True,
            "photo_size_gb": 1.2,
            "photo_duration_s": 60
        })
    with open("solver_output.yaml", "w") as f:
        yaml.dump(output, f)

def describe_solver_output(llm):
    with open("solver_output.yaml", "r") as f:
        solver_data = f.read()
    response = llm.chat.completions.create(
        messages=[{"role": "system", "content": OUTPUT_CREATOR_PROMPT},
                  {"role": "user", "content": solver_data}],
        model="OpenAI.chatgpt-4o-latest",
        temperature=0.5
    )
    return response.choices[0].message.content

In [31]:

llm = OpenAI(
    base_url="https://epita.open-webui.myia.io/api",
    api_key="sk-d8c0043ee83d4188bdc2aaee9566dd51")
user_request = "J'ai besoin d'une photo satellite de Paris de 100 km de large urgemment, et quand tu peux, une photo du centre ville de Montreal."
parsed_data = parse_user_request(llm, user_request)

In [58]:
generate_solver_input(parsed_data["requests"])

Location: Paris, GPS Coordinates: {'latitude': 48.8534951, 'longitude': 2.3483915}
Location: Montréal, GPS Coordinates: {'latitude': 45.5031824, 'longitude': -73.5698065}


In [61]:
simulate_solver()
summary = describe_solver_output(llm)
print(summary)

Here is a summary of the satellite observation results:

We successfully captured satellite imagery over two locations: Paris and Montréal. Each observation lasted 60 seconds and resulted in image files of approximately 1.2 gigabytes in size. Both operations were completed successfully, indicating that the satellite maintained stable positioning and favorable conditions during the captures.

I am confident in the accuracy and quality of these results based on the success status and consistent data across both locations. If there is any difference between these results and your expectations, it could be due to variables such as temporary weather conditions, satellite angle, or the specific area size covered during the observation window. Please let me know if you'd like to review or adjust any parameters for future observations.
